In [87]:
import sys
import os
import pandas as pd
import re
from Bio import SeqIO
from Bio.Seq import Seq
from Bio import AlignIO
from Bio.Align import AlignInfo
import zipfile
import tarfile
from collections import defaultdict
import uuid
import subprocess
import shutil

In [88]:
input_multifile = 'Des_ex3.zip'
input_csv = 'startery.csv'
min_length = 100
expected_len = 176
substitution_error_threshold = 1.56
min_amplicon_seq_frequency = 38.11
min_dominant_frequency_threshold = 43.64

p1 = f'ampliSAS_analysis'
os.makedirs(p1)

p2 = f'ampliSAS_analysis/input'
os.makedirs(p2)

if tarfile.is_tarfile(input_multifile) or zipfile.is_zipfile(input_multifile) != True:
    raise Exception('File is not a multifile')
            
if os.path.isfile(input_csv) != True:
    raise Exception('CSV file not existing')
    
primer_file = pd.read_csv(input_csv, names=[0,1])

forward = primer_file.iloc[1, 1]
reverse = str(Seq(primer_file.iloc[2, 1]).reverse_complement())
        
ID = primer_file.iloc[0, 0]
forward_ID = f'{ID}_forward'
reverse_ID = f'{ID}_reverse'

p3 = f'ampliSAS_analysis/input/fasta_primers'
os.mkdir(p3) 
   
outpath_f = f'{p3}/starter_f.fasta'
outpath_r = f'{p3}/starter_r.fasta'
with open(outpath_f, 'w') as outfile:
    outfile.write('>' + forward_ID + '\n' + forward)
with open(outpath_r, 'w') as outfile:
    outfile.write('>' + reverse_ID + '\n' + reverse)

In [89]:
#os.mkdir(f'{p2}/fastqs')
print(f'Extracting fastqs from multifile')

#for i in input_multifile:
with zipfile.ZipFile(input_multifile, 'r') as zip:
    zip.extractall(path = f'{p2}')
        
print(f'Extraction finished')

Extracting fastqs from multifile
Extraction finished


In [90]:
fastqs_folder = re.sub('\.zip', '', input_multifile)

os.rename(f'{p2}/{fastqs_folder}', f'{p2}/fastqs')

In [91]:
class Amplicon:
    def __init__(self, number):
        self.number = number
        self.clusters = None
        self.seqs_count = None
        
        
    def cut_primers(self, input_fastq, outputs_dir_good, outputs_dir_bad, starter_f, starter_r):
        command = f'python3 cutPrimers.py -r1 {input_fastq} -pr15 {starter_f} -pr13 {starter_r} -tr1 {outputs_dir_good}/amplicon{self.number}_primerfree.fastq -utr1 {outputs_dir_bad}/odrzucone_ampli{self.number}.fastq'        
        process = subprocess.check_output(command, shell=True)
        
            
    def parse_sequence_file(self, inputfastq_primerfree):
        global amplicon_table
        amplicon_table = pd.DataFrame(columns = ['Hash', 'ID', 'Sequence', 'Depth', 'Length', 'Frequency'])      ## ogarnąc czy przypadkiem nie trzeba zrobić w inicie  
        
        dedup_records = defaultdict(list) #tworzy słownik, dla których wartościami jest lista

        for record in SeqIO.parse(inputfastq_primerfree, "fastq"):
            dedup_records[str(record.seq)].append(record.id) #tworzy słownik, dla którego key=seq, a value=id. Na tym etapie dochodzi do odnalezienia duplikatów - jest ich tyle wartości id
        for seq, ids in sorted(dedup_records.items(), key=lambda t: len(t[1]), reverse=True): #ta linijka służy przesortowaniu sekwencji pod względem ich głębokości                
            id_hold = ids[0]
            id_len = len(ids)
            seq_len = len(seq)
            unique_hash = str(uuid.uuid1())
            amplicon_table = amplicon_table.append({'Hash' : unique_hash, 'ID' : id_hold, 'Sequence' : seq, 'Depth' : id_len, 'Length' : seq_len}, ignore_index = True)
        
        self.seqs_count = len(amplicon_table) # Ilość sekwencji w amplikonie
        depth_of_amplicon = amplicon_table['Depth'].sum() #Wylicza głębie całego amplikonu
        
        for index, row in amplicon_table[ ['Hash', 'ID', 'Sequence', 'Depth', 'Length', 'Frequency'] ].iterrows():
            freq_in_amplicon = row.Depth/depth_of_amplicon*100
            row.Frequency_per_amplicon = freq_in_amplicon
            
        
        return amplicon_table

        
        
    
    def create_fasta(self, inputs_dir):
        outpath = f'{inputs_dir}/{self.number}_amplicon.fasta'
        with open(outpath, 'w') as outfile:
            for index, row in amplicon_table[ ['Hash', 'ID', 'Sequence', 'Depth', 'Length', 'Frequency'] ].iterrows():
                outfile.write('>' + row.Hash + ' | ' + row.ID + ' | depth: ' + str(row.Depth) + ' | length: ' + str(row.Length) + ' | frequency per amplicon: ' + str(row.Frequency) + '\n' + row.Sequence + '\n')
                
                
    #Niesprawdzone
    def create_seq_fasta(self, seqs_fasta_dir):
        for index, row in amplicon_table[ ['Hash', 'ID', 'Sequence', 'Depth', 'Length', 'Frequency'] ].iterrows():
            outpath = f'{seqs_fasta_dir}/{row.Hash}.fasta'
            with open(outpath, 'w') as outfile:
                outfile.write('>' + row.Hash + ',' + row.ID + ',' + str(row.Depth) + ',' + str(row.Length) + ',' + str(row.Frequency) + '\n' + row.Sequence + '\n')

    #Koniec niesprawdzonego

In [164]:
class Cluster:
    def __init__(self, amplicon, min_length, expected_len):
        self.dominant_seq = None
        self.amplicon = amplicon
        self.min_length = min_length
        self.expected_len = expected_len
        
        
    def find_dominant(self):
#        amplicon_table.sort_values('Lenght', ascending=False, inplace=True)
        amplicon_table.drop(amplicon_table[amplicon_table.Length < self.min_length].index, inplace=True)  #odrzuca sekwencje o długości niższej niż zadana przez użytkownika
        amplicon_table.reset_index(drop=True, inplace=True)
        
        
        global cluster_dom
        cluster_dom = pd.DataFrame(columns = ['Hash', 'ID', 'Sequence', 'Depth', 'Length', 'Frequency'])
#        if amplicon_table[amplicon_table.Length == self.expected_length].iloc[0]:
#            dominant_seq = amplicon_table.iloc[0]
#        else:
        for index, row in amplicon_table[ ['Hash', 'ID', 'Sequence', 'Depth', 'Length', 'Frequency'] ].iterrows():
            if row.Length == self.expected_len:
                dominant_seq = amplicon_table.iloc[0]
                cluster_dom = cluster_dom.append( [dominant_seq] )
                amplicon_table.drop(0)
                amplicon_table.reset_index(drop=True, inplace=True)
                break
                       
                
    def seq2seq(self, seq1, seq2, substitution_error_threshold, temp_dir, number):
        command = f'./fogsaa {seq1} {seq2} 1 0 1 -1 -1'
        process = subprocess.check_output(command, shell=True) #Przykładowy wynik: b'176\n176\nElapsed time: 1 milliseconds\ntotal nodes expanded==176\n\nscore= 174\n'
        process = process.decode('utf-8') #Zamiana bajtów na str

        outpath = f'{temp_dir}/{number}.txt'
        with open(outpath, 'w') as outfile:
            outfile.write(process)
                
        score = int(re.sub('score=\ ', '', ' '.join(map(str, re.findall('score=\ .[0-9]*', process))))) # Wyodrębnienie score dla alignmentu
        len_seqs = int(re.sub('score=\ ', '', ' '.join(map(str, re.findall('(?<=[0-9]\s)[0-9]+', process))))) # Wyodrębnia długość porównywanych sekwencji - założenie, że porównywane sekwencje są tylko tej samej długości
        
        # Duży problem a propos wyliczenia substitution_error. Fogsaa nie zwraca jaka część score to kara za mismatch.
        # Oznacza to, że nie wiem ile substytucji tak naprawdę zaszło, a bez tego nie mogę wyliczyć błędu substytucji.
        
        global substitution_error
        substitution_error = (len_seqs - score)/len_seqs*100 # Wylicza poziom błedów dla danego alignmentu
        
        
    def multiple_aline_seqs(self, cluster_df, clusters_dir, mafftout_dir, number):
        #Tworzenie wspólnej fasty dla klastra
        outpath = f'{clusters_dir}/{number}_cluster.fasta'
        with open(outpath, 'w') as outfile:
            for index, row in cluster_df[ ['Hash', 'ID', 'Sequence', 'Depth', 'Length', 'Frequency'] ].iterrows():
                outfile.write('>' + row.Hash + ' | ' + row.ID + ' | depth: ' + str(row.Depth) + ' | length: ' + str(row.Length) + ' | frequency per amplicon: ' + str(row.Frequency) + '\n' + row.Sequence + '\n')
                        
        #Mafft na pliku --> nie jest potrzebny jesli do klastra trafiają tylko sekwencje o tej samej długości
       # command = f'mafft --auto --quiet {outpath} > {mafftout_dir}'
       # process = subprocess.check_output(command, shell=True)
        
        # Stworzenie dataframe na sekwencję konsensusową z danymi
        global consensus_df
        
        consensus_df = pd.DataFrame(columns = ['ID', 'Sequence', 'Depth', 'Length'])
        id_cluster = f'cluster_{number}'
        len_cluster = cluster_df.loc[0, 'Length']
        depth_cluster = cluster_df['Depth'].sum()
               
        #Ustalenie konsensusowej
        align = AlignIO.read(outpath, 'fasta')
        summary_align = AlignInfo.SummaryInfo(align)
        consensus = str(summary_align.dumb_consensus())
    #    my_pssm = summary_align.pos_specific_score_matrix(consensus,chars_to_ignore = ['N']) # Matrix z częstością dla danej pozycji 
    
        consensus_df = consensus_df.append({'ID' : id_cluster, 'Depth' : depth_cluster, 'Length' : len_cluster, 'Sequence' : consensus}, ignore_index = True)

In [93]:
p4 = f'{p2}/fastqs'
p5 = f'{p2}/fastqs_primerfree'
p6 = f'{p2}/fastqs_primerfree/out'
os.mkdir(p5)
os.mkdir(p6)

In [94]:
#Wyciananie starterów
files = 0
for filename in os.scandir(p4):
    files = files+1
    if filename.is_file():
        go = Amplicon(files)
        go.cut_primers(filename.path, p5, p6, outpath_f, outpath_r)
        


In [95]:
shutil.rmtree(p6)

In [165]:
p7 = f'{p2}/fasta'
os.mkdir(p7)

p8 = f'{p1}/amplicons_filtered_seqs' # Tu będa znajdowały się foldery dla każdego amplikonu zawierające każdą sekwencję w amplikonie jako osobny plik fasta. Sekwencje te są już przefiltrowane względem długości, głebi itd
os.mkdir(p8)

p9 = f'{p1}/clusters'
os.mkdir(p9)

p10 = f'{p1}/clusters/mafftout'
os.mkdir(p10)

temp_dir = f'{p1}/blad'
os.mkdir(temp_dir)

In [108]:
# min_length = 100
# expected_len = 176
# substitution_error_threshold = 1.56
# min_amplicon_seq_frequency = 38.11
# min_dominant_frequency_threshold = 43.64

In [166]:
files = 0
for filename in os.scandir(p5):
    files = files+1
    if filename.is_file():
        go = Amplicon(files)
        go.parse_sequence_file(filename.path)
        go.create_fasta(p7) #SPRAWDZONE
        do = Cluster(files, min_length, expected_len)   #Wczytanie pliku z funkcjami klastrowania
        do.find_dominant()
        seqs_fasta_dir = f'{p8}/amplicon_{files}'
        os.mkdir(seqs_fasta_dir)
        go.create_seq_fasta(seqs_fasta_dir)
        clusters_dir = f'{p9}/amplicon_{files}'
        os.mkdir(clusters_dir)
        mafftouts = f'{p10}/amplicon_{files}'
        os.mkdir(mafftouts)
        all_clusters = pd.DataFrame(columns = ['ID', 'Sequence', 'Depth', 'Length'])
        subdominants = 0 #zmienna do pilnowania liczby klastrów poza dominującym
        subdominants_df =  pd.DataFrame(columns = ['Subdominant', 'Hash', 'ID', 'Sequence', 'Depth', 'Length', 'Frequency']) #Tworzy dataframe, w którym będzie zapisywana każda sekwencja subdominująca z danego ampilkonu
        
        #Pierwsze przejście - dla sekwencji domminującej
        for index, row in amplicon_table[ ['Hash', 'ID', 'Sequence', 'Depth', 'Length', 'Frequency'] ].iterrows():
            #if index != 0: #po dodaniu dropa w find_dominant() niepotrzebne
                seq1a = cluster_dom.loc[0, 'Hash']
                seq1 = f'{seqs_fasta_dir}/{seq1a}.fasta'
                seq2 = f'{seqs_fasta_dir}/{row.Hash}.fasta'           
                do.seq2seq(seq1, seq2, substitution_error_threshold, temp_dir, index)
                if substitution_error < substitution_error_threshold:
                    #Warunek freq
                    freq_to_dom = row.Depth/cluster_dom.loc[0, 'Depth']*100
                    if row.Frequency < min_amplicon_seq_frequency or freq_to_dom < min_dominant_frequency_threshold:
                        if row.Length == cluster_dom.loc[0, 'Length']:
                            cluster_dom = cluster_dom.append(amplicon_table.iloc[index]) #dodaje sekwencję do klastra dominującego
                            amplicon_table.drop(index) #usuwa sekwencję z głównego dataframe amplikonu
                            do.multiple_aline_seqs(cluster_dom, clusters_dir, mafftouts, 'dom')
                    else:
                        subdominants += 1
                        subdominants_df = subdominants_df.append(amplicon_table.iloc[index])
                        subdominants_df = subdominants_df.append({'Subdominant' : subdominants}, ignore_index = True)
                        amplicon_table.drop(index)
        all_clusters = all_clusters.append(consensus_df.iloc[0])
        # Na tym kończy się pierwsze przejście przez amplikon. Jego efektem jest wytworzenie klastra,
        # który jako podstawe ma sekwencję dominującą w amplikonie oraz usunięcie z amplicon_table
        # zarówno sekwencji dominującej jak i pozostałych należących do klastra sekwencji.
        # Dodatkowo tworzony jest DataFrame, który przechowuje wszystkie sekwencje, które zostały
        # uznane za subdominujące i mogą zostac podstawami nowych klastrów
        
#         for index, row in subdominants_df[ ['Subdominant', 'Hash', 'ID', 'Sequence', 'Depth', 'Length', 'Frequency'] ].iterrows():
#             dom_sub_seq_cluster = pd.DataFrame(columns = ['Hash', 'ID', 'Sequence', 'Depth', 'Length', 'Frequency'])
#             dom_sub_seq = subdominants_df.iloc[0]
#             dom_sub_seq_cluster = dom_sub_seq_cluster.append( [dom_sub_seq] )
#             seq1a = dom_sub_seq.loc[0, 'Hash']
#             seq1 = f'{seqs_fasta_dir}/{seq1a}.fasta'
#             for index, row in subdominants_df[ ['Subdominant', 'Hash', 'ID', 'Sequence', 'Depth', 'Length', 'Frequency'] ].iterrows():
#                 if index != 0:
#                     if row.Length == dom_sub_seq.loc[0, 'Length']:
#                         seq2 = f'{seqs_fasta_dir}/{row.Hash}.fasta'
#                         do.seq2seq(seq1, seq2, substitution_error_threshold)
#                         if substitution_error < substitution_error_threshold:
#                             dom_sub_seq_cluster = dom_sub_seq_cluster.append(subdominants_df.iloc[index])
#                             subdominants_df.drop(subdominants_df.iloc[index])
#                             subdominants_df.reset_index(drop=True, inplace=True)
#             for index, row in amplicon_table[ ['Hash', 'ID', 'Sequence', 'Depth', 'Length', 'Frequency'] ].iterrows():
#                 dom_sub_seq = dom_sub_seq_cluster.iloc[0]
        
        #Stworzenie ostatecznego pliku fasta z sekwencjami konsensusowymi dla danego klastra
        
        outpath = f'{clusters_dir}/consensus_seqs.fasta'
        with open(outpath, 'w') as outfile:
            for index, row in all_clusters[ ['ID', 'Sequence', 'Depth', 'Length'] ].iterrows():
                outfile.write('>' + row.ID + ' | depth: ' + str(row. Depth) + ' | length: ' + str(row.Length) + '\n' + row.Sequence + '\n')
        

In [62]:
# for index, row in amplicon_table[ ['Hash', 'ID', 'Sequence', 'Depth', 'Length'] ].iterrows():
#     if index != 0:
#         seq1a = cluster_dom.loc[0, 'Hash']
#         print(seq1a)
#         #print(amplicon_table.iloc[index])